In [1]:
from resnet import *
from fashionmnist_net import *
from cnn_fmnist import *
from resnet_fmnist import *
from fashionmnist_dataset import *    
import torch as t 
import numpy as np

import sys
from numpy import polyfit
from numpy import polyval
import tqdm
import matplotlib.pylab as plt
import matplotlib.cm as cm
import json
import hyperparams
from importlib import reload
from scipy.interpolate import interp1d
from PIL import Image
from fashionmnist_funcs import accuracy

In [2]:
len(fashionmnist_loader(batch_size=128, use_aug=True))

2

In [3]:
train_loader,  test_loader = fashionmnist_loader(batch_size=128, use_aug=True)

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Resnet

In [7]:
!wget "https://github.com/passalis/pkth/blob/master/cifar10/models/resnet18_cifar10.model?raw=true"

--2021-09-07 11:50:09--  https://github.com/passalis/pkth/blob/master/cifar10/models/resnet18_cifar10.model?raw=true
Распознаётся github.com (github.com)… 140.82.121.4
Подключение к github.com (github.com)|140.82.121.4|:443... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа… 302 Found
Адрес: https://github.com/passalis/pkth/raw/master/cifar10/models/resnet18_cifar10.model [переход]
--2021-09-07 11:50:09--  https://github.com/passalis/pkth/raw/master/cifar10/models/resnet18_cifar10.model
Повторное использование соединения с github.com:443.
HTTP-запрос отправлен. Ожидание ответа… 302 Found
Адрес: https://raw.githubusercontent.com/passalis/pkth/master/cifar10/models/resnet18_cifar10.model [переход]
--2021-09-07 11:50:09--  https://raw.githubusercontent.com/passalis/pkth/master/cifar10/models/resnet18_cifar10.model
Распознаётся raw.githubusercontent.com (raw.githubusercontent.com)… 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Подключение к raw.githubusercontent.com

In [5]:
epoch_num = 100
# epoch_num = 50

run_num = 1 # количество запусков эксперимента

# версия нужна, чтобы различать старые и новые результаты экспериментов. 
# менять нужно каждый раз, когда есть хотя бы незначительные изменения в эксперименте
exp_ver = '2'

validate_every_epoch = 5 

# train_splines_every_epoch = 5 # каждые 5 эпох отслеживать траекторию гиперпараметров
# train_splines_every_epoch = 2
# train_splines_every_epoch = 3
# train_splines_every_epoch = 10

# размер мини-эпохи в батчах, за которую у нас производится либо обучение спайлов, либо их использование
# mini_epoch_size = 10

In [7]:
# teacher = ResNet18(10).to(device)
# teacher.load_state_dict(t.load('resnet18_cifar10.model?raw=true', map_location=device), )
# teacher.eval()

class_num=10
seed=42
tr_load=train_loader
t_load=test_loader
validate_every_epoch=validate_every_epoch

crit = nn.CrossEntropyLoss()

np.random.seed(seed)
t.manual_seed(seed)

for _ in range(run_num):
    internal_results = []

    teacher = ResNet18(10).to(device)
    
    

    for e in range(epoch_num):
        if e == 0 :
            optim = t.optim.Adam(teacher.parameters(), lr=1e-3)    
        if e == 50:
            optim = t.optim.Adam(teacher.parameters(), lr=1e-4)    
            
        tq = tqdm.tqdm(tr_load)
        losses = []

        for batch_id, (x,y) in enumerate(tq):
            x = x.to(device)
            y = y.to(device)

            teacher.zero_grad()           
            out = teacher(x)
            teacher_loss = crit(out, y)                                                      
            loss = teacher_loss

            losses.append(loss.cpu().detach().numpy())
            loss.backward()
            optim.step()
            tq.set_description('current loss:{}'.format(np.mean(losses[-10:])))        
            
        if e==0 or (e+1)%validate_every_epoch == 0: # если номер эпохи делится на 5 или эпоха - первая             
            test_loss = []
            teacher.eval()

            for x,y in t_load:
                x = x.to(device)
                y = y.to(device)                            
                test_loss.append(crit(teacher(x), y).detach().cpu().numpy())                 

            test_loss = float(np.mean(test_loss))
            acc = float(accuracy(teacher, t_load))
            teacher.train()

            internal_results.append({'epoch': e, 'test loss':test_loss, 'accuracy':acc})

            print (internal_results[-1])
        
    with open('../log/exp'+exp_ver+'_resnet.jsonl', 'a') as out:
        out.write(json.dumps({'results':internal_results, 'version': exp_ver})+'\n')

current loss:0.5522573590278625:   1%|          | 4/469 [00:00<00:12, 37.45it/s]

{'epoch': 0, 'test loss': 0.542353093624115, 'accuracy': 0.8021000027656555}


current loss:0.37445077300071716:   1%|          | 4/469 [00:00<00:12, 36.82it/s]

{'epoch': 4, 'test loss': 0.4211340844631195, 'accuracy': 0.8445999622344971}


current loss:0.42427515983581543:  52%|█████▏    | 246/469 [00:06<00:05, 37.85it/s]


KeyboardInterrupt: 

In [15]:
with open('../log/exp'+exp_ver+'_resnet.jsonl', 'a') as out:
    out.write(json.dumps({'results':internal_results, 'version': exp_ver})+'\n')

In [16]:
# torch.save(teacher.state_dict(), 'fmnist_teacher.cpk')

In [2]:
teacher = ResNet18(10).to('cuda')
teacher.load_state_dict(torch.load('fmnist_teacher.cpk'))

<All keys matched successfully>

In [5]:
correct = 0
total = 0
with t.no_grad():
    for x,y in train_loader_no_augumentation:
        x = x.to(device)
        y = y.to(device)
        out = teacher(x)
        correct += t.eq(t.argmax(out, 1), y).sum()
        total+=len(x)
print ('teacher accuracy on train', correct/total)        
        
correct = 0
total = 0
with t.no_grad():
    for x,y in test_loader:
        x = x.to(device)
        y = y.to(device)
        out = teacher(x)
        correct += t.eq(t.argmax(out, 1), y).sum()
        total+=len(x)
print ('teacher accuracy on test', correct/total)        
        
                

NameError: name 'teacher' is not defined

In [3]:
train_loader,  test_loader = fashionmnist_loader(batch_size=128, use_aug=False)

In [5]:
device = 'cuda'
logits = []
with t.no_grad():
    for x,y in train_loader:
        x = x.to(device)        
        out = teacher(x)
        logits.append(out.cpu().detach().numpy())
         

In [6]:
np.save('logits_resnet_fmnist', np.vstack(logits), allow_pickle=False)

# CNN

In [ ]:
!wget https://github.com/passalis/pkth/blob/master/cifar10/models/aux_pkt.model?raw=true

In [4]:
teacher = FMNIST_CNN(10).to(device)
# teacher.load_state_dict(t.load('aux_pkt.model?raw=true', map_location=device), )

In [7]:
class_num=10
seed=42
tr_load=train_loader_no_augumentation
t_load=test_loader
validate_every_epoch=validate_every_epoch
lr0=1.0

crit = nn.CrossEntropyLoss()
np.random.seed(seed)
t.manual_seed(seed)

for _ in range(run_num):
    internal_results = []

    teacher = FMNIST_CNN(class_num).to(device)
    optim = t.optim.SGD(teacher.parameters(), lr=lr0)    
    scheduler = t.optim.lr_scheduler.StepLR(optim, step_size=10, gamma=0.5)   

    for e in range(epoch_num):
        tq = tqdm.tqdm(tr_load)
        losses = []

        for batch_id, (x,y) in enumerate(tq):
            x = x.to(device)
            y = y.to(device)

            teacher.zero_grad()           
            out = teacher(x)
            teacher_loss = crit(out, y)                                                      
            loss = teacher_loss

            losses.append(loss.cpu().detach().numpy())
            loss.backward()
            optim.step()
            tq.set_description('current loss:{}'.format(np.mean(losses[-10:])))        

        if e==0 or (e+1)%validate_every_epoch == 0: # если номер эпохи делится на 5 или эпоха - первая             
            test_loss = []
            teacher.eval()

            for x,y in t_load:
                x = x.to(device)
                y = y.to(device)                            
                test_loss.append(crit(teacher(x), y).detach().cpu().numpy())                 

            test_loss = float(np.mean(test_loss))
            acc = float(accuracy(student, t_load))
            teacher.train()

            internal_results.append({'epoch': e, 'test loss':test_loss, 'accuracy':acc})

            print (internal_results[-1])
        scheduler.step()
    with open('../log/exp'+exp_ver+'_cnn.jsonl', 'a') as out:
        out.write(json.dumps({'results':internal_results, 'version': exp_ver})+'\n')

current loss:0.6560705900192261:  39%|███▉      | 185/469 [00:13<00:20, 14.16it/s]


KeyboardInterrupt: 

In [ ]:
correct = 0
total = 0
with t.no_grad():
    for x,y in train_loader_no_augumentation:
        x = x.to(device)
        y = y.to(device)
        out = teacher(x)
        correct += t.eq(t.argmax(out, 1), y).sum()
        total+=len(x)
print ('teacher accuracy on train', correct/total)        
        
correct = 0
total = 0
with t.no_grad():
    for x,y in test_loader:
        x = x.to(device)
        y = y.to(device)
        out = teacher(x)
        correct += t.eq(t.argmax(out, 1), y).sum()
        total+=len(x)
print ('teacher accuracy on test', correct/total)        
        
                

In [ ]:
logits = []
with t.no_grad():
    for x,y in train_loader_no_augumentation:
        x = x.to(device)        
        out = teacher(x)
        logits.append(out.cpu().detach().numpy())
         

In [ ]:
np.save('logits_cnn_fmnist', np.vstack(logits), allow_pickle=False)